<a href="https://www.kaggle.com/code/baharehsamadi/jdclassification-breezydb-2?scriptVersionId=136803124" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# With tf-idf
#Bahare Samadi_Adoc TM

!pip install jsonlines
!pip install pandas
!pip install spacy
!pip install langdetect
!pip install -U scikit-learn

import jsonlines
import pandas as pd
import spacy
from langdetect import detect
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup


# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Read the positions data from the CSV file
positions_df = pd.read_csv('/kaggle/input/positionscsv/positions.csv')

# Drop the row if the education is empty
positions_df = positions_df.dropna(subset=["education"])
positions_df = positions_df.reset_index(drop=True)

# Drop the row if the job description is not in English
positions_df['language'] = positions_df['description'].apply(detect)
positions_df = positions_df[positions_df['language'] == 'en']
positions_df = positions_df.drop(columns=['language'])

# Remove HTML tags from the job description
positions_df['description'] = positions_df['description'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text(separator=" "))


# Remove newline characters from the job descriptions
positions_df['description'] = positions_df['description'].str.replace('\n', '', regex=False)

# Remove the word 'Adoc Talent Management' from the job descriptions to avoid the word 'management' to be counted
positions_df['description'] = positions_df['description'].str.replace('Adoc Talent Management', '')

# Save the modified DataFrame to a CSV file
positions_df.to_csv('modified_positions.csv', index=False)

# Load the keywords from the JSON file
keywords = []
with jsonlines.open('/kaggle/input/jz-skill-patternsmaincore/jz_skill_patternsMain.jsonl') as json_file:
    for data in json_file:
        if data['label'] == 'COREPHD':
            patterns = data.get('pattern', [])
            for pattern in patterns:
                keyword = pattern.get('LOWER')
                if keyword:
                    keywords.append(keyword)

# Function to extract keywords from job description
def extract_keywords(job_description, keywords):
    doc = nlp(job_description.lower())
    found_keywords = []
    for keyword in keywords:
        if keyword in doc.text:
            found_keywords.extend([keyword] * doc.text.count(keyword))
    return found_keywords

# Create a list to store the job information
job_info = []

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Iterate over the positions
for _, row in positions_df.iterrows():
    job_id = row['_id']
    job_name = row['name']
    job_description = row['description']
    job_education = row['education']
    
    # Skip the job if the description is empty or missing
    if pd.isnull(job_description) or job_description == 'No job description found to import from MindMatch.':
        continue
    
    # Extract the keywords from the job description
    found_keywords = extract_keywords(job_description, keywords)
    
    # Calculate the tf-idf scores
    job_corpus = [job_description]
    keyword_corpus = [' '.join(found_keywords)]
    tfidf_matrix = vectorizer.fit_transform(job_corpus + keyword_corpus)
    tfidf_scores = tfidf_matrix.toarray()[0, 1:]  # Exclude the job description
    
    # Calculate the match percentage
    match = round(tfidf_scores.sum() / tfidf_scores.shape[0] * 100, 1)
    
    # Write the keywords to a text file
    with open(f'{job_id}_phds.txt', 'w') as txt_file:
        txt_file.write(f'Job Name: {job_name}\n')
        for keyword in found_keywords:
            txt_file.write(keyword + '\n')
    
    # Print the match percentage
    print(f"Job ID: {job_id}, Job_name: {job_name}, Match: {match}%, Num Words: {len(job_description.split())}")
    
    # Add job information to the list
    job_info.append({'Job ID': job_id, 'Name': job_name, 'Education': job_education, 'Match': match, 'Num Words': len(job_description.split()), 'Keywords': ', '.join(found_keywords)})

# Create a DataFrame from the job information
job_info_df = pd.DataFrame(job_info)

# Specify the path and filename for the Excel file
excel_file_path = '/kaggle/working/ResultsTF_IDF.xlsx'

# Write the DataFrame to an Excel file
job_info_df.to_excel(excel_file_path, index=False)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993241 sha256=97f4aec74a0a64e7b6d978628219eef26d2a20771da72a5ca5a58e81859fadcf
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Job ID: 8e7d7a00fb6701, Job_name: rest, Match: 5.4%, Num Words: 255
Job ID: 4b09219bb22d01, Job_name: Lead Scientist - Artificial Intelligence, Match: 8.1%, Num Words: 175
Job ID: 0564017952b801, Job_name: Mechatronics and Software Development Scientist H/F, Match: 5.8%, Num Words: 251
Job ID: c97c41c72de801, Job_name: Lead Software Developer H/F, Match: 5.9%, Num Words: 249
Job ID: 0286a158f05f01, Job_name: Researcher in Advanced Education, Match: 6.0%, Num Words: 272
Job ID: d2840269c20b01, Job_name: ANTIBODY ENGINEERING (SENIOR) SCIENTIST, Match: 6.6%, Num Words: 193
Job ID: a901ad66059901, Job_name: Biostatistician, Consulting Services (CDI) F/M, Match: 4.7%, Num Words: 300
Job ID: 7059e205499201, Job_name: Strategy and Management Consultant (CDI) F/M, Match: 3.7%, Num Words: 466
Job ID: 97a62c38f66801, Job_name: Strategy and Management Consultant (CDI) F/M, Match: 3.7%, Num Words: 466
Job ID: f0789d7c052501, Job_name: Advanced Analytics Specialist (CDI) F/M, Match: 3.7%, Num Words